In [377]:
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.remote.webelement import WebElement
from webdriver_manager.chrome import ChromeDriverManager
from typing import List, Tuple, Generator

In [378]:
# Instantiate a Chrome driver.
def load_driver() -> None:
    # Load and configure webdriver.
    options: Options = Options()
    # Stop browser windows from actually popping up.
    options.add_argument('--headless')
    # Install a browser for use by Selenium.
    service: Service = Service(executable_path=ChromeDriverManager().install())
    return Chrome(service=service, options=options)

In [379]:
# Helper function that navigates to the search result page for a particular
# month on the USCF page for historical tournament data.
def navigate_to_uscf_page(driver: Chrome, date_to_visit: str) -> None:
    # Navigate to US Chess tournament search page.
    USCF_URL = 'http://www.uschess.org/datapage/events-rated.php'
    driver.get(USCF_URL)
    date_search_box: WebElement = driver.find_element('name', 'month')
    date_search_box.clear()
    date_search_box.send_keys(date_to_visit)

    # Select CA as the State Code, which is where all 
    # chess.com USCF tournaments are registered.
    state_search_box: WebElement = driver.find_element('name', 'states')
    state_search_box.clear()
    state_search_box.send_keys('CA')
    state_search_box.submit()

In [380]:
# Scrapes tournment urls for the month that the driver is currently
# pointed add. Helper function navigate_to_uscf_page() navigates to the
# correct page.
def scrape_uscf_tournament_urls(driver: Chrome) -> List[str]:
    table_body: List[WebElement] = driver.find_elements(By.TAG_NAME, 'tbody')[2]
    table_body_row: List[WebElement] = table_body.find_elements(By.TAG_NAME, 'tr')
    url_list: List[str] = []
    for row in table_body_row:
        # Each row is a WebElement with data about one tournament.
        table_row: List[WebElement] = row.find_elements(By.TAG_NAME, 'td')
        if len(table_row) >= 3:
            url: str = None
            for element in table_row:
                if element.text.isnumeric() and len(element.text) > 10:
                    url = element.find_element(By.TAG_NAME, 'a').get_attribute('href')
                # Only keep tournament urls labeled "CHESS.COM"
                if 'CHESS.COM' in element.text.upper():
                    url_list.append(url)
    return url_list     

In [381]:
import sqlite3
from sqlite3 import Connection, Cursor

# Drop then create tables for all data scraped by this module.
def init_db() -> None:
    conn: Connection = sqlite3.Connection('scrape_data.db')
    cur: Cursor = conn.cursor()
    cur.execute("""DROP TABLE IF EXISTS uscf_urls""")
    # Stores urls to allow us to navigate to all relevant USCF tournaments.
    cur.execute("""CREATE TABLE IF NOT EXISTS uscf_urls (
        id INTEGER PRIMARY KEY,
        date TEXT,
        url TEXT,
        scraped INTEGER
        )""")
    conn.commit()

In [382]:
# Scrapes a list of urls. Each url is a USCF tournament with a parallel entry
# on chess.com. Every month between 2015 and 2023 inclusive is checked.
def scrape_all_uscf_urls(driver: Chrome, cur: Cursor) -> List[str]:
    url_list: List[str] = []
    for year in range(2023, 2014, -1):
        # Page requires single digit months to have a 0 in front.
        for month in range(1, 10):
            date: str = '0' + str(month) + '/' + str(year)
            navigate_to_uscf_page(driver, date)
            url_list.extend(scrape_uscf_tournament_urls(driver))
        for month in range(10, 13):
            date: str = str(month) + '/' + str(year)
            navigate_to_uscf_page(driver, date)
            url_list.extend(scrape_uscf_tournament_urls(driver))
    return url_list

### TOURNAMENT OBSERVATIONS
INTEGER PRIMARY KEY
Round count
Player count
Number of rounds
Section date(s)
Received date
Entered date
Rated date
Re-Rated date
K Factor
Rating System
Tournament type
Time control
Tournament name
Tournament ID
Total points

### ROUNDS RESULT OBSERVATIONS
INTEGER PRIMARY KEY
TOURNAMENT FOREIGN KEY
Round number
Round result
Round opponent

### PLAYER OBSERVATIONS
INTEGER PRIMARY KEY
Real name
USCF before rating
USCF after rating
State
Pair number
Total points
USFC id


In [383]:
# from datetime import datetime

# STARTING_URL: str = 'https://www.chess.com/tournament/live?&page='

# def go_to_tournament(day, month, year, starting_url):
#     driver.get(starting_url)
#     date_string = driver.find_element(By.CLASS_NAME, 'tournaments-live-date')
#     date = datetime.strptime(date_string.text, '%b %d, %Y, %I:%M %p')

# 0 to 9000 is what chess.com currently allows
# def go_to_right_year(target, url, page) -> bool:
#     driver.get(url + page)
#     date_string = driver.find_element(By.CLASS_NAME, 'tournaments-live-date')
#     date = datetime.strptime(date_string.text, '%b %d, %Y, %I:%M %p')
#     if target < date.year:
#         page = page + step_size
#         step_size = step_size / 2

In [384]:
# driver: Chrome = load_driver()
# url_list: List[str] = scrape_all_uscf_urls(driver)

# url_tuples: List[Tuple] = []
# for url in url_list:
#     url_tuples.append((None, url, 0))

# conn: Connection = sqlite3.Connection('scrape_data.db')
# cur: Cursor = conn.cursor()
# cur.executemany("""INSERT INTO uscf_urls (
#     date, url, scraped
#     ) VALUES (?, ?, ? )""", url_tuples)
# conn.commit()

In [385]:
from bs4 import BeautifulSoup
from bs4.element import ResultSet, Tag
import requests
from requests.models import Response

# Currently set to only return data from one tourament page.
def get_tournament_data(tournament_id: str) -> BeautifulSoup:
    cur: Cursor = sqlite3.Connection('scrape_data.db').cursor()
    cur.execute("SELECT url FROM uscf_urls WHERE id = ?", (tournament_id,))
    url: str = cur.fetchall()[0][0]
    request: Response = requests.get(url)
    print(url)
    print('Status Code:', request.status_code)
    return BeautifulSoup(request.text, 'html.parser')

In [386]:
# Extracts the following in order: tournament_name, tournament_id, event_date, received_date,
# entered_date, rated_date, section_count, player_count
def extract_tournament_info1(soup: BeautifulSoup) -> List[str]:
    upper_table: ResultSet[Tag] = soup.find_all('table', attrs={
        'border': '0',
        'bgcolor': 'FFFFFF',
        'cellpadding': '3',
        'cellspacing': '0'
    })
    rows: ResultSet[Tag] = upper_table[0].find_all('tr')
    row1_tags: ResultSet[Tag] = rows[0].find_all('td')

    infoset1: List[str] = []
    infoset1.append(row1_tags[3].b.text)
    infoset1.append(row1_tags[3].small.text[1:-1])
    infoset1.append(row1_tags[7].b.text)

    dates_split: List[str] = row1_tags[13].b.text.split(' ')
    infoset1.append(dates_split[1])
    infoset1.append(dates_split[4])
    infoset1.append(dates_split[7])
    infoset1.append(row1_tags[15].b.text.split(' ')[0])
    infoset1.append(row1_tags[15].b.text.split(' ')[-2])
    return infoset1

In [387]:
# Extracts the following in order: k_factor, rating_system, tournament_type, time_control
def extract_tournament_info2(soup: BeautifulSoup) -> List[str]:
    upper_table: ResultSet[Tag] = soup.find_all('table', attrs={
            'border': '0',
            'bgcolor': 'FFFFFF',
            'cellpadding': '3',
            'cellspacing': '0'
        })
    header_box: Tag = upper_table[1]
    rules: str = header_box.find_all('b')[3].text
    rules_list: List[str] = rules.split(' ')

    infoset2: List[str] = []
    infoset2.append(rules_list[7])
    infoset2.append(rules_list[11])
    infoset2.append(rules_list[16])
    infoset2.append(rules_list[20])
    return infoset2


In [388]:
import re
from re import Match

# Includes both player names and links. Each link needs to be
# preceded by https://www.uschess.org/msa/.
def get_player_names_and_urls(soup: BeautifulSoup) -> ResultSet[Tag]:
    results: ResultSet[Tag] = soup.find_all('a', {'href': lambda x: x and x.startswith('MbrDtlMain')})
    players: List[Tuple[str]] = []
    for result in results:
        players.append((result.text, 'https://www.uschess.org/msa/' + result['href']))
    return players

def extract_tabular_results(soup: BeautifulSoup) -> List[str]:
    pre_results: ResultSet[Tag] = soup.find_all('pre')
    # Using .stripped_strings returns the data not surrounded by an HTML tag, which is the
    # results data we want.
    results_raw: Generator = pre_results[0].stripped_strings
    results_clean: List[Tuple[str]] = []
    results_query: str = r'\|\d+\.\d+\s*(?:\|[A-Z]\s*\d*)+'
    state_query: str = r'\|\s+[A-Z][A-Z]\s+\|'
    rating_query: str = r'(\|\s*\d+\s*\/\s*[A-Z]+:[A-Za-z\s0-9]+->[A-Za-z\s0-9]+)([\sBW\|]*)'

    player_rows: List[Tuple[str]] = []
    for string in results_raw:
        score: List[str] = re.findall(results_query, string)
        state: List[str] = re.findall(state_query, string)
        ratings_and_pairings = re.findall(rating_query, string)
        
        if score or state or ratings_and_pairings:
            player_rows.append((score, state, ratings_and_pairings[0][0], ratings_and_pairings[0][1]))

    clean_tabs: List[List[str]] = []
    for row in player_rows:
        player_data: List[str] = []
        # Player scores in different rounds
        if row[0]:
            split_scores: List[str] = row[0][0].split('|')
            for s in split_scores:
                s = s.strip()
                if s != '' and s != ',':
                    player_data.append(s)
        # State codes
        if row[1]:
            player_data.append(row[1][0].strip('|').strip('\n').strip(' '))
        # Rating changes during the tournament
        if row[2]:
            split_ratings: List[str] = row[2].strip('|').split('/')
            player_data.append(split_ratings[0].strip())
            second_split: List[str] = split_ratings[-1].split(':')
            player_data.append(second_split[0].strip())
            before_rating, after_rating = second_split[1].split('->')
            player_data.append(before_rating.strip())
            player_data.append(after_rating.strip())
        if row[3]:
            player_data.append(row[3].strip())
        clean_tabs.append(player_data)
    return clean_tabs

In [389]:
soup: BeautifulSoup = get_tournament_data('2001')
metadata1: List[str] = extract_tournament_info1(soup)
metadata2: List[str] = extract_tournament_info2(soup)
players: List[Tuple[str]] = get_player_names_and_urls(soup)
tournament_results = extract_tabular_results(soup)

for i in metadata1:
    print(i)
for i in metadata2:
    print(i)
for i in players:
    print(i)
for i in tournament_results:
    print(i)

http://www.uschess.org/msa/XtblMain.php?201811268082
Status Code: 200
US CHESS BLITZ ON CHESS.COM #1
201811268082
2018-11-26 
2018-11-28
2018-11-28
2018-11-29
1
25
F
OB
S
G/10;+0
('RYAN DEAN KARLOFF', 'https://www.uschess.org/msa/MbrDtlMain.php?12785405')
('LUIS R VALDES SILVA', 'https://www.uschess.org/msa/MbrDtlMain.php?14715400')
('JORDAN PAGE', 'https://www.uschess.org/msa/MbrDtlMain.php?14948893')
('THOMAS BLAINE WEBB', 'https://www.uschess.org/msa/MbrDtlMain.php?12701656')
('JORDAN EVERETT EVERE TIMM', 'https://www.uschess.org/msa/MbrDtlMain.php?15078846')
('BRYAN BARNETTE', 'https://www.uschess.org/msa/MbrDtlMain.php?12755799')
('PRAKASH MAURYA', 'https://www.uschess.org/msa/MbrDtlMain.php?16734637')
('ALAN BOKIEV', 'https://www.uschess.org/msa/MbrDtlMain.php?15180467')
('ALEXANDER DAVID GONZALEZ', 'https://www.uschess.org/msa/MbrDtlMain.php?14027058')
('JACOB PETER SNELL', 'https://www.uschess.org/msa/MbrDtlMain.php?15278548')
('UDAYACHANDRU PANNEERSELVAM', 'https://www.uschess

In [390]:
# soup: BeautifulSoup = get_tournament_data('501')
# pretty_soup = soup.prettify()
# with open('page.txt', 'w') as fout:
#     for line in pretty_soup:
#         fout.write(line)

http://www.uschess.org/msa/XtblMain.php?202211234952
Status Code: 200
